In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [505]:
#Get dates and gameid for each game in 2020/2021 English Premier League

url='https://api.sportsdata.io/v3/soccer/scores/json/CompetitionDetails/EPL'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

result=pd.json_normalize(response.json())
games = pd.DataFrame(result['Games'][0])#.flatten()#.columns

games_final = games[['GameId','AwayTeamId','HomeTeamId','Day','AwayTeamName','HomeTeamName']]

Days=games_final['Day'].unique().tolist()

games_final['Day'] = pd.to_datetime(games_final['Day'])

TeamID = games_final['AwayTeamId'].unique().tolist()



<ipython-input-505-20c29b3168b6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_final['Day'] = pd.to_datetime(games_final['Day'])


In [560]:
url='https://api.sportsdata.io/v3/soccer/odds/json/GameOddsByDate/2020-09-12'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

response=response.json()
result=pd.DataFrame(response)

In [561]:
#Aggregated Odds per game 

odds=result['PregameOdds']
games=[]
for row in odds:
    if row==[]:
        continue
    else:
        games.append(row)

empty_list = []

for i in range(len(games)):
    
    empty_list.append(pd.DataFrame(games[i]))
    
games_odds=pd.concat(lis)

games=games_odds.groupby('GameId').agg({'HomeMoneyLine':'mean','AwayMoneyLine':'mean','DrawMoneyLine':'mean'})


In [562]:
#Merging with initial table

daily_games = pd.merge(left = result,
                                 right = games,
                                 how = 'right', 
                                 left_on = "GameId", 
                                 right_on= "GameId")


daily_games=daily_games[['GameId', 'RoundId','Day','Status', 
                         'AwayTeamId', 'HomeTeamId', 'AwayTeamName', 'HomeTeamName','HomeTeamScore',
                         'AwayTeamScore', 'TotalScore','HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine']]

daily_games[['Day']]=pd.to_datetime(daily_games['Day'])

#Selecting games only from EPL
#daily_games=daily_games[daily_games['RoundId']==602]

In [563]:
daily_games

,GameId,RoundId,Day,Status,AwayTeamId,HomeTeamId,AwayTeamName,HomeTeamName,HomeTeamScore,AwayTeamScore,TotalScore,HomeMoneyLine,AwayMoneyLine,DrawMoneyLine
0,30971,579,2020-09-12,Final,877,992,CD Guadalajara,Club Necaxa,2.0,4.0,6.0,299.500000,172.500000,313.500000
1,30972,579,2020-09-12,Final,889,1570,Club Puebla,FC Juárez,2.0,0.0,2.0,153.000000,360.000000,310.000000
2,31799,590,2020-09-12,Final,915,1559,FK Ufa,FK Tambov,4.0,0.0,4.0,230.000000,268.000000,261.000000
3,32481,602,2020-09-12,Final,509,1462,Arsenal FC,Fulham FC,0.0,6.0,6.0,625.666667,-212.833333,418.333333
4,31798,590,2020-09-12,Final,2370,916,FK Khimki,FK Ural,6.0,2.0,8.0,189.000000,325.000000,265.000000
5,32486,602,2020-09-12,Final,520,512,Southampton FC,Crystal Palace FC,2.0,0.0,2.0,298.333333,206.500000,319.833333
6,34227,644,2020-09-12,Final,622,628,RC Celta de Vigo,SD Eibar,0.0,0.0,0.0,250.500000,264.500000,278.500000
7,30528,578,2020-09-12,Final,901,900,OGC Nice,Montpellier HSC,6.0,2.0,8.0,155.000000,346.000000,332.000000
8,31797,590,2020-09-12,Final,1560,917,PFK Sochi,RFK Akhmat Grozny,0.0,2.0,2.0,202.000000,275.000000,289.000000
9,32482,602,2020-09-12,Final,2148,515,Leeds United FC,Liverpool FC,8.0,6.0,14.0,-529.000000,1482.000000,730.500000
